In [4]:
import os
import glob
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Tuple

## Funções Auxiliares (Carregamento, Salvamento e Visualização)

In [5]:
def desserializar_keypoints(arr: np.ndarray) -> List[cv2.KeyPoint]:
    """
    Reconstrói lista de cv2.KeyPoint a partir do array (N,7).
    """
    kps = []
    for row in arr:
        k = cv2.KeyPoint(
            x=float(row[0]), y=float(row[1]), size=float(row[2]),
            angle=float(row[3]), response=float(row[4]),
            octave=int(row[5]), class_id=int(row[6])
        )
        kps.append(k)
    return kps


def carregar_features_npz(caminho_npz: str) -> Tuple[List[cv2.KeyPoint], np.ndarray, str]:
    """
    Lê um .npz salvo pela Etapa 2 e retorna (keypoints, descriptors, imagem_absoluta).
    """
    with np.load(caminho_npz, allow_pickle=True) as data:
        kps = desserializar_keypoints(data["keypoints"])
        desc = data["descriptors"]
        img_path = str(data["imagem_absoluta"])
    return kps, desc, img_path


def carregar_matches_npz(caminho_npz) -> List[cv2.DMatch]:
    """
    Carrega os matches salvos em .npz e restaura cada linha como um objeto cv2.DMatch.
    Retorna uma lista de cv2.DMatch.
    """

    with np.load(caminho_npz, allow_pickle=True) as data:
        arr = data["matches"]  # array shape (N, 3)
        matches = []
        for row in arr:
            # cv2.DMatch(queryIdx, trainIdx, distance)
            m = cv2.DMatch(_queryIdx=int(row[0]), _trainIdx=int(row[1]), _distance=float(row[2]))
            matches.append(m)
    return matches


def calcular_homogr(kps1, kps2, matches, params = None, direction = '2to1'):
    """
    Calcula a homografia entre duas imagens usando os pontos correspondentes.
    """
    if not kps1 or not kps2 or not matches or len(matches) < 4:
        return None, None

    # Extrai os pontos-chave e descritores
    pts1 = [kps1[m.queryIdx].pt for m in matches]
    pts2 = [kps2[m.trainIdx].pt for m in matches]

    src, dst = (pts2, pts1) if direction == "2to1" else (pts1, pts2)

    # Calcula a homografia usando RANSAC
    H, mask = cv2.findHomography(np
                                 .array(src, dtype=np.float32), np.array(dst, dtype=np.float32), cv2.RANSAC, **(params or {}))

    return H, mask

def alinhar_imgs(img1, img2, homografia):
    if homografia is None:
        return None
    
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    # Cantos da imagem 2
    cantos2 = np.array([[0,0],[w2,0],[w2,h2],[0,h2]], dtype=np.float32).reshape(-1,1,2)

    # Aplica a homografia nos cantos da imagem 2
    cantos2_trans = cv2.perspectiveTransform(cantos2, homografia)

    # Combina com os cantos da imagem 1 para calcular o retangulo final
    cantos_totais = np.vstack((cantos2_trans, np.array([[0,0],[w1,0],[w1,h1],[0,h1]], dtype=np.float32).reshape(-1,1,2)))
    xmin, ymin = np.floor(cantos_totais.min(axis=0).ravel()).astype(np.int32)
    xmax, ymax = np.ceil(cantos_totais.max(axis=0).ravel()).astype(np.int32)

    # Translacao para coordenadas positivas
    translacao = [-xmin, -ymin]
    T = np.array([[1,0,translacao[0]], [0,1,translacao[1]], [0,0,1]], dtype=np.float64)

    # Aplica warpPerspective com a homografia ajustada
    res = cv2.warpPerspective(img2, T @ homografia, (xmax - xmin, ymax - ymin))

    # Coloca img1 na posição correta
    res[translacao[1]:translacao[1]+h1, translacao[0]:translacao[0]+w1] = img1

    return res

def salvar_homogr_npz(dir_saida, i, j, H, mask, path1, path2, metrics, params=None, direction="2to1"):
    os.makedirs(dir_saida, exist_ok=True)
    mask_arr = np.asarray(mask, dtype=np.uint8).reshape(-1,1) if mask is not None else np.zeros((0,1), np.uint8)
    np.savez_compressed(
        os.path.join(dir_saida, f"homogr_{i:03d}_{j:03d}.npz"),
        H=np.asarray(H, dtype=np.float32) if H is not None else np.zeros((3,3), np.float32),
        mask=mask_arr,
        inliers=np.int32(metrics.get("inliers", 0)),
        total=np.int32(metrics.get("total", 0)),
        rmse_px=np.float32(metrics.get("rmse_px") if metrics.get("rmse_px") is not None else np.nan),
        direction=str(direction),                 # "2to1" => img2 -> img1
        params_json=json.dumps(params or {}),     # p/ reprodutibilidade
        idx_i=np.int32(i), idx_j=np.int32(j),
        path_i=str(path1), path_j=str(path2)
    )

def carregar_homogr_npz(caminho_npz):
    """
    Lê homogr_*.npz e retorna:
      H (3x3 float32), mask (Nx1 uint8), meta (dict: inliers, total, rmse_px, direction, params_json, idx_i, idx_j, path_i, path_j)
    """
    with np.load(caminho_npz, allow_pickle=True) as d:
        H = d["H"].astype(np.float32)
        mask = d["mask"].astype(np.uint8)
        meta = {
            "inliers": int(d["inliers"]),
            "total": int(d["total"]),
            "rmse_px": None if np.isnan(d["rmse_px"]) else float(d["rmse_px"]),
            "direction": str(d["direction"]),
            "params_json": str(d["params_json"]),
            "idx_i": int(d["idx_i"]), "idx_j": int(d["idx_j"]),
            "path_i": str(d["path_i"]), "path_j": str(d["path_j"]),
        }
    return H, mask, meta

def carregar_homogr_dir(dir_homogr):
    arquivos = sorted(glob.glob(os.path.join(dir_homogr, "homogr_*.npz")))
    dados = []
    for f in arquivos:
        H, mask, meta = carregar_homogr_npz(f)
        dados.append((H, mask, meta, f))
    return dados  # lista de tuplas (H, mask, meta, caminho)

def _rmse_reproj(H, pts_src, pts_dst, mask):
    if H is None or mask is None or not mask.any():
        return None
    sel = mask.ravel().astype(bool)
    if sel.sum() < 4:
        return None
    src = np.float32(pts_src)[sel]  # pts_src no sistema da imagem de origem em H
    dst = np.float32(pts_dst)[sel]  # pts_dst no sistema de destino
    src_h = cv2.convertPointsToHomogeneous(src).reshape(-1,3).T
    proj = (H @ src_h); proj = (proj[:2] / proj[2]).T
    dif = dst - proj
    return float(np.sqrt((dif**2).sum(axis=1).mean()))

# CHANGED
def listar_pares_de_matches(caminho_matcher_dir, only_sequential: bool = True, n_imgs: int | None = None):
    """
    Retorna lista de tuplas (i, j, caminho_npz_ou_None, reversed_bool).
    - only_sequential=True: exatamente os pares (i, (i+1)%n), na ordem, incluindo wrap (n-1, 0).
    - only_sequential=False: retorna todos os matches_*.npz encontrados como (i,j,path,False).
    """
    arquivos = sorted(glob.glob(os.path.join(caminho_matcher_dir, "matches_*.npz")))
    parsed = []
    for arq in arquivos:
        base = os.path.splitext(os.path.basename(arq))[0]  # matches_000_001
        parts = base.split("_")
        if len(parts) < 3:
            continue
        _, si, sj = parts[:3]
        parsed.append((int(si), int(sj), arq))

    if not only_sequential:
        # todos os arquivos existentes, sem reversed (pois já estão no sentido do nome do arquivo)
        return [(i, j, path, False) for (i, j, path) in parsed]

    # --- sequenciais com wrap ---
    if n_imgs is None:
        idxs = set()
        for i, j, _ in parsed:
            idxs.update([i, j])
        n = max(idxs) + 1 if idxs else 0
    else:
        n = int(n_imgs)

    out = []
    for i in range(n):
        j = (i + 1) % n
        path, reversed_flag = _resolve_caminho_matches(caminho_matcher_dir, i, j)
        out.append((i, j, path, bool(reversed_flag)))
    return out




# RANSAC em cascata (do mais rígido ao menos rígido)  # NEW
RANSAC_CANDIDATOS_RIGOROSOS = [
    {"ransacReprojThreshold": 1.0, "maxIters": 22000, "confidence": 0.9997},
    {"ransacReprojThreshold": 1.4, "maxIters": 20000, "confidence": 0.9995},
    {"ransacReprojThreshold": 1.8, "maxIters": 18000, "confidence": 0.9990},
]

def _rmse_simetrico(H, pts2, pts1, mask):  # NEW
    if H is None or mask is None or not np.any(mask):
        return None
    sel = mask.ravel().astype(bool)
    if sel.sum() < 4:
        return None
    p1 = np.float32(pts1)[sel]
    p2 = np.float32(pts2)[sel]
    try:
        Hinv = np.linalg.inv(H)
    except np.linalg.LinAlgError:
        return None
    def _proj(P, M):
        Ph = cv2.convertPointsToHomogeneous(P).reshape(-1,3).T
        Q  = (M @ Ph); Q = (Q[:2] / Q[2]).T
        return Q
    e_fwd = np.linalg.norm(p1 - _proj(p2, H),   axis=1)  # 2->1
    e_bwd = np.linalg.norm(p2 - _proj(p1, Hinv), axis=1) # 1->2
    e = 0.5*(e_fwd + e_bwd)
    return float(np.sqrt((e**2).mean()))

def _homografia_saudavel_bbox(H, shape_src, shape_dst, fator_max_bbox=6.0):  # NEW
    if H is None or not np.isfinite(H).all(): return False
    h2, w2 = shape_src[:2]
    if h2 <= 0 or w2 <= 0: return False
    cantos2 = np.float32([[0,0],[w2,0],[w2,h2],[0,h2]]).reshape(-1,1,2)
    try:
        proj = cv2.perspectiveTransform(cantos2, H)
    except cv2.error:
        return False
    if not np.isfinite(proj).all(): return False
    xs = proj[:,0,0]; ys = proj[:,0,1]
    bw = float(xs.max() - xs.min()); bh = float(ys.max() - ys.min())
    if bw <= 0 or bh <= 0: return False
    h1, w1 = shape_dst[:2]
    diag_ref  = max((h1*h1 + w1*w1) ** 0.5, (h2*h2 + w2*w2) ** 0.5)
    diag_proj = (bw*bw + bh*bh) ** 0.5
    return (diag_proj <= fator_max_bbox * float(diag_ref))

def _condicionamento_ok(H, max_kappa=1e6):  # NEW
    if H is None or not np.isfinite(H).all(): return False
    try:
        s = np.linalg.svd(H, compute_uv=False)
    except np.linalg.LinAlgError:
        return False
    if s[-1] == 0: return False
    kappa = float(s[0] / s[-1])
    return (kappa <= max_kappa)

def checar_integridade_H(H, mask, pts2, pts1, img1, img2,
                         min_inliers=12, max_rmse=3.0, max_kappa=1e6, fator_max_bbox=6.0):
    """
    Retorna (ok: bool, metrics: dict, reason: str|None)
      metrics: 'inliers', 'total', 'rmse_px', 'rmse_sym', 'kappa'
      reason: motivo textual da reprova (para visualização/debug; NÃO é salvo)
    """
    metrics = {"inliers": 0, "total": len(pts1), "rmse_px": None, "rmse_sym": None, "kappa": None}

    # 1) Existência
    if H is None or mask is None:
        return False, metrics, "sem H ou mask"

    inl = int(mask.sum())
    metrics["inliers"] = inl

    # 2) Inliers mínimos
    if inl < min_inliers:
        return False, metrics, f"poucos inliers ({inl} < {min_inliers})"

    # 3) RMSE direcional (2->1) e simétrico
    try:
        rmse_dir = _rmse_reproj(H, pts2, pts1, mask)  # se já existir no arquivo
    except NameError:
        sel = mask.ravel().astype(bool)
        if sel.sum() >= 4:
            src = np.float32(pts2)[sel]; dst = np.float32(pts1)[sel]
            src_h = cv2.convertPointsToHomogeneous(src).reshape(-1,3).T
            proj = (H @ src_h); proj = (proj[:2] / proj[2]).T
            dif = dst - proj
            rmse_dir = float(np.sqrt((dif**2).sum(axis=1).mean()))
        else:
            rmse_dir = None
    metrics["rmse_px"] = rmse_dir

    rmse_sym = _rmse_simetrico(H, pts2, pts1, mask)
    metrics["rmse_sym"] = rmse_sym

    if (rmse_dir is None or rmse_dir > max_rmse) and (rmse_sym is None or rmse_sym > max_rmse):
        rd = f"{rmse_dir:.2f}" if rmse_dir is not None else "None"
        rs = f"{rmse_sym:.2f}" if rmse_sym is not None else "None"
        return False, metrics, f"RMSE alto (dir={rd}, sym={rs} > {max_rmse})"

    # 4) Condicionamento
    try:
        s = np.linalg.svd(H, compute_uv=False)
        kappa = float(s[0] / s[-1]) if s[-1] != 0 else np.inf
    except np.linalg.LinAlgError:
        kappa = np.inf
    metrics["kappa"] = kappa
    if not _condicionamento_ok(H, max_kappa=max_kappa):
        return False, metrics, f"condicionamento ruim (kappa={kappa:.2e} > {max_kappa:.1e})"

    # 5) Sanidade geométrica (bbox)
    if not _homografia_saudavel_bbox(H, img2.shape, img1.shape, fator_max_bbox=fator_max_bbox):
        return False, metrics, "bbox projetada exagerada"

    # OK
    return True, metrics, None


# CHANGED: agora aceita fator_max_bbox e usa título compactado
def visualizar_par(img1, img2, H, titulo="Pré-visualização do par", metrics=None, bbox_factor=None):
    if img1 is None or img2 is None or H is None:
        return
    h1, w1 = img1.shape[:2]
    warped = cv2.warpPerspective(img2, H, (w1, h1))
    blend  = cv2.addWeighted(img1, 0.6, warped, 0.4, 0)

    # cantos projetados de img2 no frame de img1
    h2, w2 = img2.shape[:2]
    cant2  = np.float32([[0,0],[w2,0],[w2,h2],[0,h2]]).reshape(-1,1,2)
    poly   = cv2.perspectiveTransform(cant2, H).reshape(-1,2)

    # calcula bbox_factor se não vier de fora
    if bbox_factor is None:
        bbox_factor = _bbox_factor(H, img2.shape, img1.shape)

    plt.figure(figsize=(16,8))
    plt.imshow(cv2.cvtColor(blend, cv2.COLOR_BGR2RGB))
    xs = np.r_[poly[:,0], poly[0,0]]; ys = np.r_[poly[:,1], poly[0,1]]
    plt.plot(xs, ys, 'r-', lw=2)
    if metrics:
        txt = _format_preview_metrics(metrics, bbox_factor=bbox_factor)
        plt.title(f"{titulo}\n{txt}")
    else:
        plt.title(titulo)
    plt.axis('off'); plt.show()



# NEW helper: formatação compacta para o título do preview
def _format_preview_metrics(metrics, bbox_factor=None):
    import math
    def f2(x):
        if x is None or (isinstance(x, float) and (math.isnan(x) or math.isinf(x))):
            return "—"
        return f"{x:.2f}"
    def fexp(x):
        if x is None or (isinstance(x, float) and (math.isnan(x) or math.isinf(x))):
            return "—"
        return f"{x:.2e}"
    inl   = metrics.get("inliers", "—")
    rmse  = f2(metrics.get("rmse_px"))
    rsym  = f2(metrics.get("rmse_sym"))
    kappa = fexp(metrics.get("kappa"))
    extra = f", bbox={bbox_factor:.2f}×diag" if (bbox_factor is not None) else ""
    return f"inliers={inl}, rmse={rmse}, rmse_sym={rsym}, kappa={kappa}{extra}"


def _bbox_factor(H, shape_src, shape_dst):
    """
    Retorna o fator (diag_bbox_proj / diag_ref), onde:
      - diag_bbox_proj = diagonal do retângulo mínimo que contém os 4 cantos da src projetados por H no frame dst
      - diag_ref = max(diagonal da src, diagonal da dst)
    Se não for possível calcular, retorna None.
    """
    if H is None or not np.isfinite(H).all():
        return None
    h2, w2 = shape_src[:2]
    h1, w1 = shape_dst[:2]
    if h2 <= 0 or w2 <= 0 or h1 <= 0 or w1 <= 0:
        return None

    cantos2 = np.float32([[0,0],[w2,0],[w2,h2],[0,h2]]).reshape(-1,1,2)
    try:
        proj = cv2.perspectiveTransform(cantos2, H)
    except cv2.error:
        return None
    if not np.isfinite(proj).all():
        return None

    xs = proj[:,0,0]; ys = proj[:,0,1]
    bw = float(xs.max() - xs.min()); bh = float(ys.max() - ys.min())
    if bw <= 0 or bh <= 0:
        return None

    diag_proj = (bw*bw + bh*bh) ** 0.5
    diag_src  = (h2*h2 + w2*w2) ** 0.5
    diag_dst  = (h1*h1 + w1*w1) ** 0.5
    diag_ref  = max(diag_src, diag_dst)
    if diag_ref <= 0:
        return None

    return float(diag_proj / diag_ref)

# NEW: encontra o arquivo de matches do par (i,j); se só existir (j,i), marca como reversed=True
def _resolve_caminho_matches(caminho_matcher_dir: str, i: int, j: int):
    p_ij = os.path.join(caminho_matcher_dir, f"matches_{i:03d}_{j:03d}.npz")
    if os.path.exists(p_ij):
        return p_ij, False
    p_ji = os.path.join(caminho_matcher_dir, f"matches_{j:03d}_{i:03d}.npz")
    if os.path.exists(p_ji):
        return p_ji, True
    return None, None

# NEW: inverte query/train dos DMatch quando o arquivo é (j,i) mas queremos (i,j)
def _inverter_matches(matches: List[cv2.DMatch]) -> List[cv2.DMatch]:
    inv = []
    for m in matches:
        inv.append(cv2.DMatch(_queryIdx=m.trainIdx, _trainIdx=m.queryIdx, _distance=float(m.distance)))
    return inv



## Pipeline Principal da Etapa 4

In [6]:
# CHANGED: função principal com preview exibindo fator_max_bbox e métricas formatadas
def analisar_homogr_alinhamento(caminho_base_etapa2: str, caminho_base_etapa3: str, detectores: List[str], matchers: List[str],
                                ransac_cfg=None, gerar_preview: bool = True, only_sequential: bool = True,
                                validate_H: bool = True, validation_cfg: dict | None = None):
    """
    Etapa 4: (re)calcula homografias e salva homogr_*.npz.

    Parâmetros:
      - only_sequential: True => processa apenas pares sequenciais com wrap (i, (i+1)%n);
                          False => processa todos os pares encontrados em matches_*.npz.
      - validate_H: ativa validação avançada (inliers / RMSE / condicionamento / bbox).
      - validation_cfg: dict opcional para thresholds e/ou lista 'candidatos' de RANSAC.
          chaves aceitas:
            min_inliers (int), max_rmse (float), max_kappa (float),
            fator_max_bbox (float), candidatos (list[dict])
      - ransac_cfg: utilizado SOMENTE quando validate_H=False (uma única rodada de RANSAC).
      - gerar_preview: exibe visualização por par (não salva imagens).
    """
    print(f"\n{'='*65}\nHomografia e alinhamento para o conjunto: '{os.path.basename(caminho_base_etapa3)}'\n{'='*65}")

    # Carrega ordem (apenas para logs/consistência)
    caminho_json = os.path.join(caminho_base_etapa2, "ordem_imagens.json")
    try:
        with open(caminho_json, "r") as f:
            arquivos_de_imagem_abs = json.load(f)
    except FileNotFoundError:
        print(f"[Erro] 'ordem_imagens.json' não encontrado em '{caminho_base_etapa2}'. Abortando.")
        return

    # Config de validação
    vcfg = validation_cfg or {}
    min_inl    = int(vcfg.get("min_inliers", 12))
    max_rmse   = float(vcfg.get("max_rmse", 3.0))
    max_kappa  = float(vcfg.get("max_kappa", 1e6))
    fator_bbox = float(vcfg.get("fator_max_bbox", 6.0))
    candidatos = vcfg.get("candidatos", RANSAC_CANDIDATOS_RIGOROSOS)

    # Itera detectores
    for nome_detector in detectores:
        print(f"\n--- Detector: {nome_detector.upper()} ---")
        caminho_detector = os.path.join(caminho_base_etapa2, nome_detector)
        arquivos_feature_npz = sorted(glob.glob(os.path.join(caminho_detector, "*_features.npz")))
        n_imgs = len(arquivos_feature_npz)
        if n_imgs < 2:
            print("  [Aviso] Menos de 2 imagens com features; pulando.")
            continue

        # Itera matchers
        for nome_matcher in matchers:
            print(f"  - {nome_detector.upper()} / {nome_matcher.upper()}")
            caminho_matcher = os.path.join(caminho_base_etapa3, nome_detector, nome_matcher)
            dir_saida = os.path.join("resultados_etapa4", os.path.basename(caminho_base_etapa3), nome_detector, nome_matcher)
            os.makedirs(dir_saida, exist_ok=True)

            # CHANGED: montar pares sequenciais com wrap usando os arquivos já existentes
            pares = listar_pares_de_matches(
                caminho_matcher,
                only_sequential=True if only_sequential else False,
                n_imgs=n_imgs
            )
            if not pares:
                print("    [Aviso] Nenhum matches_*.npz encontrado.")
                continue


            # CHANGED: logo no começo do loop de pares, antes de carregar matches
            # CHANGED: agora recebe também reversed_flag
            for (i, j, caminho_npz_matches, reversed_flag) in pares:
                # Garantia de processar o wrap: se faltar arquivo, loga e segue
                if caminho_npz_matches is None:
                    print(f"    [Aviso] Arquivo de matches ausente para par {i:03d}-{j:03d} (wrap incluído).")
                    continue
                
                # Carrega features / imagens
                kps1, desc1, path1 = carregar_features_npz(arquivos_feature_npz[i])
                kps2, desc2, path2 = carregar_features_npz(arquivos_feature_npz[j])
                img1 = cv2.imread(path1); img2 = cv2.imread(path2)
            
                # Carrega matches do arquivo
                match_data = carregar_matches_npz(caminho_npz_matches)
            
                # Se o arquivo estava no sentido (j,i), invertimos os índices para alinhar com (i,j)
                if reversed_flag:
                    match_data = _inverter_matches(match_data)
            
                if not kps1 or not kps2 or not match_data:
                    salvar_homogr_npz(dir_saida, i, j, None, None, path1, path2,
                                      metrics={"inliers":0,"total":0,"rmse_px":None},
                                      params={}, direction="2to1")
                    if gerar_preview:
                        visualizar_par(img1, img2, None, titulo=f"Par {i}-{j} • sem H (sem matches suficientes)")
                    continue

                # ... (resto da função permanece igual)


                # Pontos 2->1
                pts1 = [kps1[m.queryIdx].pt for m in match_data]  # destino (img1)
                pts2 = [kps2[m.trainIdx].pt for m in match_data]  # origem  (img2)

                if validate_H:
                    H_ok = None; M_ok = None; metrics_ok = None; cfg_ok = None
                    reason_last = None; metrics_last = None
                    for cfg in candidatos:
                        H_try, M_try = calcular_homogr(kps1, kps2, match_data, cfg, '2to1')
                        ok, metrics, reason = checar_integridade_H(
                            H_try, M_try, pts2, pts1, img1, img2,
                            min_inliers=min_inl, max_rmse=max_rmse,
                            max_kappa=max_kappa, fator_max_bbox=fator_bbox
                        )
                        if ok:
                            H_ok, M_ok, metrics_ok, cfg_ok = H_try, M_try, metrics, cfg
                            break
                        reason_last = reason
                        metrics_last = metrics

                    if H_ok is not None:
                        if gerar_preview:
                            bf = _bbox_factor(H_ok, img2.shape, img1.shape)  # NEW
                            visualizar_par(
                                img1, img2, H_ok,
                                titulo=f"Par {i}-{j} • Homografia VÁLIDA",
                                metrics=metrics_ok,
                                bbox_factor=bf  # CHANGED
                            )

                        salvar_homogr_npz(dir_saida, i, j, H_ok, M_ok, path1, path2,
                                          metrics={"inliers":metrics_ok["inliers"],
                                                   "total":metrics_ok["total"],
                                                   "rmse_px":metrics_ok["rmse_px"]},
                                          params=cfg_ok, direction="2to1")
                    else:
                        if gerar_preview and 'H_try' in locals() and H_try is not None:
                            tit = f"Par {i}-{j} • Homografia REPROVADA — {reason_last}"
                            bf  = _bbox_factor(H_try, img2.shape, img1.shape)  # NEW
                            visualizar_par(
                                img1, img2, H_try,
                                titulo=tit,
                                metrics=metrics_last if metrics_last is not None else {},
                                bbox_factor=bf  # CHANGED
                            )

                        salvar_homogr_npz(dir_saida, i, j, None, None, path1, path2,
                                          metrics={"inliers":0,"total":len(match_data),"rmse_px":None},
                                          params=candidatos[-1] if candidatos else {}, direction="2to1")

                else:
                    # Caminho simples: uma rodada de RANSAC
                    H, M = calcular_homogr(kps1, kps2, match_data, ransac_cfg or {}, '2to1')
                    inliers = int(M.sum()) if M is not None else 0
                    rmse    = _rmse_reproj(H, pts2, pts1, M) if (H is not None and M is not None) else None
                    if gerar_preview and H is not None:
                        bf = _bbox_factor(H, img2.shape, img1.shape)  # NEW
                        visualizar_par(
                            img1, img2, H,
                            titulo=f"Par {i}-{j} • H sem validação",
                            metrics={"inliers":inliers,"total":len(match_data),"rmse_px":rmse},
                            bbox_factor=bf  # CHANGED
                        )

                    salvar_homogr_npz(dir_saida, i, j, H, M, path1, path2,
                                      metrics={"inliers":inliers,"total":len(match_data),"rmse_px":rmse},
                                      params=ransac_cfg or {}, direction="2to1")


## Execução do Pipeline para os Conjuntos de Imagens

In [ ]:
RAIZ_ETAPA2 = "resultados_etapa2"
RAIZ_ETAPA3 = "resultados_etapa3"
DETECTORS = ['sift', 'orb', 'akaze']
MATCHERS = ['bf', 'flann']
ONLY_SEQUENTIAL = True
GERAR_PREVIEW = True
VALIDATE_H = True

conjuntos = [d for d in os.listdir(RAIZ_ETAPA2) if os.path.isdir(os.path.join(RAIZ_ETAPA2, d))]
#conjuntos = ["images-developed-tiff-16bit","images-developed-tiff-8bit"]

validation_cfg = {
    "min_inliers": 8,
    "max_rmse": 1,
    "max_kappa": 3e6,
    "fator_max_bbox": 1.6,
    "candidatos": [  # opcional: substitui RANSAC_CANDIDATOS_RIGOROSOS
         {"ransacReprojThreshold": 0.75, "maxIters":30000, "confidence": 0.9999},
         {"ransacReprojThreshold": 0.9, "maxIters": 24000, "confidence": 0.9997},
         {"ransacReprojThreshold": 1.3, "maxIters": 20000, "confidence": 0.9995},
         {"ransacReprojThreshold": 1.8, "maxIters": 18000, "confidence": 0.9990},
    ]
}

ransac_cfg = validation_cfg["candidatos"][0] if "candidatos" in validation_cfg else None

for nome_conjunto in sorted(conjuntos):
    caminho_detector = os.path.join(RAIZ_ETAPA2, nome_conjunto)
    caminho_matcher = os.path.join(RAIZ_ETAPA3, nome_conjunto)
    analisar_homogr_alinhamento(caminho_detector, caminho_matcher, DETECTORS, MATCHERS, ransac_cfg, GERAR_PREVIEW,ONLY_SEQUENTIAL, VALIDATE_H, validation_cfg)

## Seleção de melhores homografias

### Definição

In [8]:
import os, glob, json, math
import numpy as np

def compilar_melhores_homografias(
    raiz_etapa4: str = "resultados_etapa4",
    raiz_etapa2: str = "resultados_etapa2",
    formatos: list[str] | None = None,
    detectores: list[str] | None = None,
    matchers: list[str] | None = None,
    nome_detector_out: str = "best",
    nome_matcher_out: str = "best",
    usar_pares_sequenciais: bool = True,
):
    """
    Para cada <formato> em resultados_etapa4, cria um diretório <formato>/best/best/
    com a melhor homografia por par (escolhida entre todos os (detector,matcher)),
    de forma compatível com a Etapa 5.

    Critério:
      1) Maior número de inliers (meta['inliers'])
      2) Desempate por menor rmse_px (meta['rmse_px']) — NaN perde no desempate.

    Se um par não tiver nenhuma H válida, salva sentinela inválida (H/mask None)
    e agora **emite um warning**.
    """
    if formatos is None:
        formatos = [d for d in sorted(os.listdir(raiz_etapa4))
                    if os.path.isdir(os.path.join(raiz_etapa4, d))]

    for formato in formatos:
        dir_formato_et4 = os.path.join(raiz_etapa4, formato)
        if not os.path.isdir(dir_formato_et4):
            continue

        caminho_ordem = os.path.join(raiz_etapa2, formato, "ordem_imagens.json")
        try:
            with open(caminho_ordem, "r") as f:
                caminhos_imgs = json.load(f)
        except FileNotFoundError:
            print(f"[Aviso] '{caminho_ordem}' não encontrado — pulando formato '{formato}'.")
            continue

        n = len(caminhos_imgs)
        if n < 2:
            print(f"[Aviso] Formato '{formato}' com menos de 2 imagens — pulando.")
            continue

        dets = detectores or [d for d in sorted(os.listdir(dir_formato_et4))
                              if os.path.isdir(os.path.join(dir_formato_et4, d))]
        dir_out = os.path.join(raiz_etapa4, formato, nome_detector_out, nome_matcher_out)
        os.makedirs(dir_out, exist_ok=True)

        candidatos_por_par: dict[tuple[int,int], list[str]] = {}

        if usar_pares_sequenciais:
            pares = [(i, (i+1) % n) for i in range(n)]
            for det in dets:
                dir_det = os.path.join(dir_formato_et4, det)
                if not os.path.isdir(dir_det): continue
                mats = matchers or [m for m in sorted(os.listdir(dir_det))
                                    if os.path.isdir(os.path.join(dir_det, m))]
                for mat in mats:
                    dir_mat = os.path.join(dir_det, mat)
                    for (i, j) in pares:
                        p = os.path.join(dir_mat, f"homogr_{i:03d}_{j:03d}.npz")
                        if os.path.exists(p):
                            candidatos_por_par.setdefault((i,j), []).append(p)
            # NEW: garante chave para todos os pares sequenciais (mesmo sem arquivos)
            for (i, j) in pares:  # NEW
                candidatos_por_par.setdefault((i, j), [])  # NEW
        else:
            for det in dets:
                dir_det = os.path.join(dir_formato_et4, det)
                if not os.path.isdir(dir_det): continue
                mats = matchers or [m for m in sorted(os.listdir(dir_det))
                                    if os.path.isdir(os.path.join(dir_det, m))]
                for mat in mats:
                    dir_mat = os.path.join(dir_det, mat)
                    for p in glob.glob(os.path.join(dir_mat, "homogr_*.npz")):
                        base = os.path.splitext(os.path.basename(p))[0]  # homogr_000_001
                        _, si, sj = base.split("_")
                        i, j = int(si), int(sj)
                        candidatos_por_par.setdefault((i,j), []).append(p)

        def _score(meta):
            inl = meta.get("inliers", 0) or 0
            rmse = meta.get("rmse_px", np.nan)
            if rmse is None or (isinstance(rmse, float) and (math.isnan(rmse) or math.isinf(rmse))):
                rmse = float("inf")
            return (-int(inl), float(rmse))

        total_pares = len(candidatos_por_par)
        if total_pares == 0:
            print(f"[Aviso] Nenhuma homografia encontrada em '{dir_formato_et4}'.")
            continue

        print(f"[Info] Compilando '{formato}': {total_pares} pares...")

        for (i, j), caminhos_npz in sorted(candidatos_por_par.items()):
            # NEW: warning se não há candidatos para o par (p.ex., arquivo inexistente)
            if not caminhos_npz:  # NEW
                print(f"[Aviso] {formato}: par {i:03d}-{j:03d} sem arquivos de homografia em nenhum método.")  # NEW
                path_i = caminhos_imgs[i]; path_j = caminhos_imgs[j]
                # salva sentinela inválida para manter compatibilidade com Etapa 5
                salvar_homogr_npz(  # NEW
                    dir_out, i, j,
                    None, None,
                    path_i, path_j,
                    metrics={"inliers":0, "total":0, "rmse_px": None},
                    params={}, direction="2to1",
                )
                continue  # NEW

            melhor = None
            melhor_meta = None
            melhor_mask = None
            melhor_params = {}
            melhor_src = None

            for caminho in caminhos_npz:
                H, mask, meta = carregar_homogr_npz(caminho)

                try:
                    parts = caminho.replace("\\","/").split("/")
                    idx_formato = parts.index(raiz_etapa4) + 1 if raiz_etapa4 in parts else None
                    if idx_formato is None:
                        idx_formato = parts.index(formato)
                    det_src = parts[idx_formato+1]
                    mat_src = parts[idx_formato+2]
                except Exception:
                    det_src, mat_src = "?", "?"

                meta = dict(meta) if meta is not None else {}
                sc = _score(meta)

                if melhor is None or sc < _score(melhor_meta or {}):
                    melhor = H
                    melhor_mask = mask
                    melhor_meta = meta
                    melhor_params = meta.get("params_json", {})
                    melhor_src = (det_src, mat_src)

            path_i = caminhos_imgs[i]
            path_j = caminhos_imgs[j]

            # CHANGED: warning explícito quando não há homografia válida (inliers<=0 ou H/mask ausentes)
            if melhor is None or melhor_mask is None or (melhor_meta or {}).get("inliers", 0) <= 0:
                print(f"[Aviso] {formato}: par {i:03d}-{j:03d} sem homografia VÁLIDA entre os métodos — salvando sentinela.")  # NEW
                salvar_homogr_npz(
                    dir_out, i, j,
                    None, None,
                    path_i, path_j,
                    metrics={"inliers":0, "total": int(melhor_meta.get("total", 0)) if melhor_meta else 0, "rmse_px": None},
                    params={},  # sem params
                    direction="2to1",
                )
            else:
                params_out = melhor_params
                if isinstance(params_out, dict):
                    params_out = dict(params_out)
                    params_out["_src_detector"] = melhor_src[0]
                    params_out["_src_matcher"]  = melhor_src[1]
                salvar_homogr_npz(
                    dir_out, i, j,
                    melhor, melhor_mask,
                    path_i, path_j,
                    metrics={
                        "inliers": int(melhor_meta.get("inliers", 0) or 0),
                        "total":   int(melhor_meta.get("total", 0)   or 0),
                        "rmse_px": (None if ("rmse_px" not in melhor_meta) else melhor_meta.get("rmse_px")),
                    },
                    params=params_out,
                    direction="2to1",
                )

        print(f"[OK] Compilado criado em: {dir_out}\n"
              f"     Use na Etapa 5 com detector='{nome_detector_out}', matcher='{nome_matcher_out}'.")


### Execução

In [9]:
RAIZ_ETAPA2 = "resultados_etapa2"
RAIZ_ETAPA3 = "resultados_etapa3"
RAIZ_ETAPA4 = "resultados_etapa4"
DETECTORS = ['sift', 'orb', 'akaze']
MATCHERS = ['bf', 'flann']

compilar_melhores_homografias(
    raiz_etapa4=RAIZ_ETAPA4,
    raiz_etapa2=RAIZ_ETAPA2,
    formatos= [d for d in os.listdir(RAIZ_ETAPA2) if os.path.isdir(os.path.join(RAIZ_ETAPA2, d))],  # todos
    detectores=DETECTORS,
    matchers=MATCHERS,
    nome_detector_out="best",
    nome_matcher_out="best",
    usar_pares_sequenciais=True,  # pares sequenciais com wrap
)


[Info] Compilando 'images-developed-png-16bit': 30 pares...
[OK] Compilado criado em: resultados_etapa4/images-developed-png-16bit/best/best
     Use na Etapa 5 com detector='best', matcher='best'.
[Info] Compilando 'images-developed-tiff-16bit': 30 pares...
[OK] Compilado criado em: resultados_etapa4/images-developed-tiff-16bit/best/best
     Use na Etapa 5 com detector='best', matcher='best'.
[Info] Compilando 'images-developed-png-8bit': 30 pares...
[OK] Compilado criado em: resultados_etapa4/images-developed-png-8bit/best/best
     Use na Etapa 5 com detector='best', matcher='best'.
[Info] Compilando 'images-pro': 22 pares...
[Aviso] images-pro: par 021-000 sem homografia VÁLIDA entre os métodos — salvando sentinela.
[OK] Compilado criado em: resultados_etapa4/images-pro/best/best
     Use na Etapa 5 com detector='best', matcher='best'.
[Info] Compilando 'images-developed-tiff-8bit': 30 pares...
[OK] Compilado criado em: resultados_etapa4/images-developed-tiff-8bit/best/best
     U

## Etapa bônus: identificação da ordem das imagens com inliers

In [9]:
import networkx as nx
import glob
import numpy as np
import os
import random

def ciclo_guloso(G, inicio, mais_leve):
    """
    Encontra um ciclo começando e terminando em 'inicio', sempre escolhendo o vizinho de menor (mais_leve=True)
    ou maior (mais_leve=False) peso, exceto o pai e já visitados, até retornar ao início ou não haver mais opções.
    """
    caminho = [inicio]
    atual = inicio
    pai = None
    visitados = set([inicio])
    while True:
        vizinhos = [v for v in G.neighbors(atual) if v != pai and v not in visitados]
        # Permite voltar ao início se todos os outros já foram visitados
        if not vizinhos and inicio in G.neighbors(atual):
            caminho.append(inicio)
            print("Ciclo completo")
            break
        if not vizinhos:
            print("Sem mais vizinhos disponíveis, ciclo incompleto")
            break
        if mais_leve:
            proximo = min(vizinhos, key=lambda v: G[atual][v]['weight'])
        else:
            proximo = max(vizinhos, key=lambda v: G[atual][v]['weight'])
        caminho.append(proximo)
        visitados.add(proximo)
        pai, atual = atual, proximo
    return caminho

def construir_grafo_de_homografias(caminho_base_etapa4: str, conjunto_imagens: str, detector: str, matcher: str, peso='inliers'):
    """
    Constrói um grafo direcionado onde cada nó é uma imagem e cada aresta é uma homografia entre imagens.
    O peso da aresta pode ser 'inliers' (número de inliers, maior é melhor) ou 'rmse' (menor é melhor).
    """
    dir_homogr = os.path.join(caminho_base_etapa4, conjunto_imagens, detector, matcher)
    arquivos_homogr = sorted(glob.glob(os.path.join(dir_homogr, "homogr_*.npz")))
    G = nx.Graph()
    
    for arq in arquivos_homogr:
        _, _, meta = carregar_homogr_npz(arq)
        i, j = meta['idx_i'], meta['idx_j']
        path_i, path_j = meta['path_i'], meta['path_j']
        inliers = meta['inliers']
        rmse = meta['rmse_px']
        
        if peso == 'inliers':
            if inliers > 0:
                G.add_edge(f"{path_i}", f"{path_j}", weight=inliers)
        elif peso == 'rmse':
            if rmse is not None and rmse > 0:
                G.add_edge(f"{path_i}", f"{path_j}", weight=rmse)
        else:
            raise ValueError("Peso deve ser 'inliers' ou 'rmse'.")
    
    return G

### Usando número de inliers

In [8]:
# Exemplo de uso com inliers como peso
caminho_base_etapa4 = "resultados_etapa4"
conjunto_imagens = "images-developed-png-8bit"
detector = "sift"
matcher = "flann"
peso = "inliers"

G = construir_grafo_de_homografias(caminho_base_etapa4, conjunto_imagens, detector, matcher, peso)

# inicio = random.choice(list(G.nodes()))   # Escolha um nó aleatório como início
inicio = list(G.nodes())[0]                 # Ou escolha um nó específico conhecido

ciclo = ciclo_guloso(G, inicio, mais_leve=False)
print(f"Ciclo guloso começando e terminando em '{inicio}':", ciclo)

# Salva o ciclo em um arquivo json
with open(os.path.join(caminho_base_etapa4, conjunto_imagens, detector, matcher, "ordem.json"), "w") as f:
    json.dump(ciclo, f)

Ciclo completo
Ciclo guloso começando e terminando em 'images-developed-png-8bit/panorama360raw-1.png': ['images-developed-png-8bit/panorama360raw-1.png', 'images-developed-png-8bit/panorama360raw-2.png', 'images-developed-png-8bit/panorama360raw-3.png', 'images-developed-png-8bit/panorama360raw-4.png', 'images-developed-png-8bit/panorama360raw-5.png', 'images-developed-png-8bit/panorama360raw-6.png', 'images-developed-png-8bit/panorama360raw-7.png', 'images-developed-png-8bit/panorama360raw-8.png', 'images-developed-png-8bit/panorama360raw-9.png', 'images-developed-png-8bit/panorama360raw-10.png', 'images-developed-png-8bit/panorama360raw-11.png', 'images-developed-png-8bit/panorama360raw-12.png', 'images-developed-png-8bit/panorama360raw-13.png', 'images-developed-png-8bit/panorama360raw-14.png', 'images-developed-png-8bit/panorama360raw-15.png', 'images-developed-png-8bit/panorama360raw-16.png', 'images-developed-png-8bit/panorama360raw-17.png', 'images-developed-png-8bit/panorama36

### Usando rmse (não está funcionando bem)

In [ ]:
# Exemplo de uso com rmse como peso
caminho_base_etapa4 = "resultados_etapa4"
conjunto_imagens = "images-developed-png-8bit"
detector = "sift"
matcher = "flann"
peso = "rmse"

G = construir_grafo_de_homografias(caminho_base_etapa4, conjunto_imagens, detector, matcher, peso)

# inicio = random.choice(list(G.nodes()))   # Escolha um nó aleatório como início
inicio = list(G.nodes())[0]                 # Ou escolha um nó específico conhecido

ciclo = ciclo_guloso(G, inicio, mais_leve=True)
print(f"Ciclo guloso começando e terminando em '{inicio}':", ciclo)

Ciclo completo
Ciclo guloso começando e terminando em 'images-developed-png-8bit/panorama360raw-11.png': ['images-developed-png-8bit/panorama360raw-11.png', 'images-developed-png-8bit/panorama360raw-4.png', 'images-developed-png-8bit/panorama360raw-16.png', 'images-developed-png-8bit/panorama360raw-21.png', 'images-developed-png-8bit/panorama360raw-10.png', 'images-developed-png-8bit/panorama360raw-23.png', 'images-developed-png-8bit/panorama360raw-17.png', 'images-developed-png-8bit/panorama360raw-6.png', 'images-developed-png-8bit/panorama360raw-28.png', 'images-developed-png-8bit/panorama360raw-9.png', 'images-developed-png-8bit/panorama360raw-22.png', 'images-developed-png-8bit/panorama360raw-8.png', 'images-developed-png-8bit/panorama360raw-18.png', 'images-developed-png-8bit/panorama360raw-12.png', 'images-developed-png-8bit/panorama360raw-5.png', 'images-developed-png-8bit/panorama360raw-24.png', 'images-developed-png-8bit/panorama360raw-13.png', 'images-developed-png-8bit/panor